In [1]:
# 测试环境 :
#     appKey：bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6
#     secretKey ：4ed8b056c32939b9fd66987470b3e9fb720bdded02197e678e516bdcdf810833
#     渠道id：1200
#     测试环境地址：http://test1.thirdpart.kaolatest.netease.com/api/接口名称
# 接口文档地址：https://github.com/kaola-thirdpart/netease-kaola-thirdpart/wiki

In [204]:
import requests
import time
import json
import os
import urllib.parse
import hashlib
# 系统参数：

# 名称	类型	是否必须	描述
# timestamp	string	Y	时间戳，格式为yyyy-mm-dd HH:mm:ss，例如：2013-05-06 13:52:03。考拉API服务端允许客户端请求时间误差为6分钟。
# app_key	string	Y	分配给应用的AppKey ，创建应用时可获得
# v	string	Y	API协议版本，可选值:1.0。
# sign	string	Y	对 API 调用参数（除sign外）进行 md5 加密获得。获取方法参考如下1.4.1
# sign_method	string	Y	参数的加密方法选择，可选值是：md5
url = 'http://test1.thirdpart.kaolatest.netease.com/api/'
channel_id = '1200'
app_key = 'bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6'
secret_key = '4ed8b056c32939b9fd66987470b3e9fb720bdded02197e678e516bdcdf810833'

def concat_params(params):
    pairs = []
    for key in sorted(params):
        if key == 'sign':
            continue
        val = params[key]
        pairs.append("{}{}".format(key, val))
    return ''.join(pairs)
def gen_sign(params, consumer_secret):
    # 根据参数名称（除签名和图片）将所有请求参数按照字母先后顺序排序:key + value .... key + value
    # 例如：将foo=1,bar=2,baz=3 排序为bar=2,baz=3,foo=1，参数名和参数值链接后，得到拼装字符串bar2baz3foo1
    # 系统同时支持MD5加密方式: md5：将secretKey 拼接到参数字符串头、尾进行md5加密后，再转化成大写，
    # 格式是：byte2hex(md5(secretKeykey1value1key2value2...secretKey)) 
    # 注：hex为自定义方法，JAVA中MD5是对字节数组加密，加密结果是16字节，我们需要的是32位的大写字符串
    params = concat_params(params)
    to_hash = hashlib.md5(bytes('{consumer_secret}{params}{consumer_secret}'.format(params=params,consumer_secret=consumer_secret),'utf-8'))
    sign = to_hash.hexdigest()
    return sign.upper()

In [165]:
def query_all_goods_id():
    """获取商品skuid列表，查询自己渠道下所有的商品skuid列表。
    接口说明：
    渠道下所有商品id查询，接口地址：queryAllGoodsId"""
    
    # channelId	String	是	渠道商ID	
    # timestamp	String	是	时间戳	
    # v	String	是	Api版本	
    # sign_method	String	是	签名方法	
    # app_key	String	是	考拉分发的appKey	
    # sign	String	是	签名
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryAllGoodsId',params=body)
    return resp.json()
query_all_goods_id()

{'goodsInfo': ['00000100460',
  '10204244-40ff49789ce88f957fcc41b900807275',
  '10220018-40ff49789ce88f957fcc41b900807275',
  '1226800-9f09660c69ecafe91e2d3a395db2968e',
  '1226800-e975675bba97f5dfaff78ce18a22464f',
  '13285677-ecc4090b639c47f89b453980923afb8e',
  '15928264-88971fe79fba958fc151a190dca3544b',
  '171563-e94a8ae8eac15e81c7286f10453bbb4f',
  '1720994-0d67186392123e7a0e2dc1806a87960e',
  '1720994-2276d9c199147d3f35de06c82769cb55',
  '1720994-c5ca4fc077a065f880e0570dd4461a0e',
  '1720994-d2b959467464fec365fdd1361ee88d30',
  '2308470-7fd974ff55d3326a4bd8e83e1de77116',
  '2308470-bc14ad1a6e79c137f2373d8f00ffd744',
  '258809-10053b3032cd03140dae78b155cba464',
  '260107-2afb4f4d30b85b7266474331c92b188a',
  '26147305-ecc4090b639c47f89b453980923afb8e',
  '26765638-4ec8f5bd59928cbb85efb15c1ca8b184',
  '26765638-6605a66e15648a893913be172d2fec55',
  '26765638-86614c59eb2d26502e5219d91e1184a5',
  '26765638-8f00d6340b92ff29fce656d7a689d48a',
  '26765638-a1e1cb77107513f4ede2eca6a0721cd6

In [137]:
resp.request.url,resp.json

('http://test1.thirdpart.kaolatest.netease.com/api/queryAllGoodsId?channelId=1200&timestamp=2018-03-19+15%3A39%3A51&v=1.0&sign_method=md5&app_key=bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6&sign=345944BD6BFD76EC5373BCB491D39F46',
 <bound method Response.json of <Response [200]>>)

In [ ]:
resp

In [166]:
# 单个商品详情查询
def query_goods_info_by_id(sku_id):
    """渠道下单个商品信息详情查询，接口地址：queryGoodsInfoById"""
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "skuId":sku_id,
        "queryType":0, # 0表示返回全部信息、1表示只返回关键信息如：商品id、价格和库存
        "channelSalePrice":'1' # 回传税后价，仅支持保税、直邮、一般贸易（税率为0）模式、海淘模式；个人物品清关模式不返回
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryGoodsInfoById',params=body)
    try:
        resp_data = resp.json()
    except:
        resp_data = resp.text
    return resp_data
query_goods_info_by_id('2308470-7fd974ff55d3326a4bd8e83e1de77116')

{'goodsInfo': {'IsPostageFree': False,
  'afterTaxPrice': '1.00',
  'brandCountryName': '中国香港',
  'brandId': 1849,
  'brandName': '网易宝贝',
  'canReturnGoods': 1,
  'category': 'DF-测试1 服装 服饰鞋包',
  'detail': '<p style="text-align:center;"><img src="http://haitao.nosdn5.127.net/in75upvd56_790_398.gif" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn1.127.net/in75uq1u44_790_1230.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn1.127.net/in75uquf52_790_859.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn1.127.net/in75ur1982_790_936.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn5.127.net/in75urlt59_790_525.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn5.127.net/in75us3782_790_861.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nosdn3.127.net/in75usrj69_790_1200.jpg" /></p>',
  'goodsId': 2308470,
  'goodsImages': [{'goodsId': 0,
    'imageType': 1,
   

In [169]:
# 商品详情批量查询
def query_goods_info_by_ids(sku_ids):
    """商品信息详情批量查询，接口地址：queryGoodsInfoByIds 由于http请求对响应时间有限制，现在默认一次只能查询20条，多于20条会返回错误码-103，详情见错误码说明"""
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "skuIds":str(sku_ids),
        "queryType":0, # 0表示返回全部信息、1表示只返回关键信息如：商品id、价格和库存
        "channelSalePrices":str([1]*len(sku_ids)) # 回传税后价，仅支持保税、直邮、一般贸易（税率为0）模式、海淘模式；个人物品清关模式不返回
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryGoodsInfoByIds',params=body)
    return resp.json()
query_goods_info_by_ids(['59070208-96359ea7676df2eea9ce9acf7c538005', '59076218-6f04aaa51ddfbe2aad2e5d139528bc95', '59077671-cdfe59eb911f5b727240c310df9fad57', '59077671-cdfe59eb911f5b727240c310df9fad57', '59083052-6f04aaa51ddfbe2aad2e5d139528bc95', '59086857-6f04aaa51ddfbe2aad2e5d139528bc95', '6014-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6024-d46389875f39726d27af6c3e81fdcf77', '6024-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6064-128010f98e698bb5365cc41a972e4b9f', '6626-bba69a3033db10d7908615a3dedeb6c5', '6720750-ecc4090b639c47f89b453980923afb8e', '6886-ecc4090b639c47f89b453980923afb8e', '7030302-ecc4090b639c47f89b453980923afb8e', '8979-3845321bfcf28ddc40b4cca6613834a3', '9017-fd619f0359020094b63a73656691297d', '9100-ecc4090b639c47f89b453980923afb8e'])

[{'goodsInfo': {'afterTaxPrice': '1.00',
   'brandCountryName': '美国',
   'brandId': 1008,
   'brandName': '三星111',
   'category': '直冲 战网 游戏点卡 虚拟商品',
   'detail': '123',
   'goodsId': 59070208,
   'goodsImages': [{'goodsId': 0,
     'imageType': 1,
     'imageUrl': 'http://pop.nosdn.127.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor430': 'http://pop.nosdn.127_430_430.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor640490': 'http://pop.nosdn.127_640_490.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor70': 'http://pop.nosdn.127_70_70.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'orderValue': 1}],
   'goodsProperty': {},
   'imageUrl': 'http://pop.nosdn.127.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
   'importType': 3,
   'isFreeShipping': 0,
   'isFreeTax': 0,
   'isPresell': 0,
   'leafCategoryId': 24656,
   'logisticsProperty': None,
   'marketPrice': 388,
   'memberCount': 0,
   'onlineStatus': 0,
   'price': 388.0,
   'productId': 59070208,
   

In [146]:
sku_ids = ['59070208-96359ea7676df2eea9ce9acf7c538005', '59076218-6f04aaa51ddfbe2aad2e5d139528bc95', '59077671-cdfe59eb911f5b727240c310df9fad57', '59077671-cdfe59eb911f5b727240c310df9fad57', '59083052-6f04aaa51ddfbe2aad2e5d139528bc95', '59086857-6f04aaa51ddfbe2aad2e5d139528bc95', '6014-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6024-d46389875f39726d27af6c3e81fdcf77', '6024-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6064-128010f98e698bb5365cc41a972e4b9f', '6626-bba69a3033db10d7908615a3dedeb6c5', '6720750-ecc4090b639c47f89b453980923afb8e', '6886-ecc4090b639c47f89b453980923afb8e', '7030302-ecc4090b639c47f89b453980923afb8e', '8979-3845321bfcf28ddc40b4cca6613834a3', '9017-fd619f0359020094b63a73656691297d', '9100-ecc4090b639c47f89b453980923afb8e']

In [155]:
sku_ids[5]

'59086857-6f04aaa51ddfbe2aad2e5d139528bc95'

In [133]:
# 商品信息增量更新接口（性能待优化）
def query_changed_goods_info():
    """根据时间戳的商品信息更新接口 queryChangedGoodsInfo;该接口限制只能查询30天之内的数据;"""
    body = {
        "source":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "startTime":int(time.time())*1000,
        "endTime":int(time.time()+86400*24)*1000
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryChangedGoodsInfo',params=body)
    return resp.json()
query_changed_goods_info()

{'recCode': 200, 'recMeg': '查询成功', 'result': []}

In [170]:
def query_comment_info():
    """查询goodsId下面指定时间段内所有的评论信息；默认查询间隔不得大于三个月(90天);接口地址queryCommentInfo;"""
    body = {
        "source":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "startTime":int(time.time())*1000,
        "endTime":int(time.time()+86400*24)*1000,
        "goodsId":59070208
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryChangedGoodsInfo',params=body)
    return resp.json()
query_comment_info()

{'recCode': 200, 'recMeg': '查询成功', 'result': []}

In [194]:
# 一、获取分类categoryId 和  title
category_list = []
brand_list = []

for item in requests.get('https://sp.kaola.com/api/category?V330').json()['body']['categoryTreeMenuList']:
    if 'categoryId' not in item:
        continue
    resp = requests.get(f'https://sp.kaola.com/api/category/frontCategory?V330&categoryId={item["categoryId"]}').json()['body']
    # 'brandList', 'level2CategoryList'
    category_tree = {'categoryId': item['categoryId'], 'categoryName': item['title'],'childCategoryViewList':resp['level2CategoryList']}
    pprint(item)
    category_list.append(category_tree)
with open('kaola_category_tree.json','w') as f:
    f.write(json.dumps(category_list,indent=2,ensure_ascii=False))

{'categoryId': 8171, 'title': '美容彩妆', 'type': 2}
{'categoryId': 8181, 'title': '奶粉纸尿裤', 'type': 2}
{'categoryId': 8182, 'title': '母婴专区', 'type': 2}
{'categoryId': 8183, 'title': '轻奢', 'type': 2}
{'categoryId': 8185, 'title': '营养保健', 'type': 2}
{'categoryId': 8184, 'title': '个人洗护', 'type': 2}
{'categoryId': 8187, 'title': '手表配饰', 'type': 2}
{'categoryId': 8188, 'title': '数码家电', 'type': 2}
{'categoryId': 8190, 'title': '运动户外', 'type': 2}
{'categoryId': 15413, 'title': '家居生活', 'type': 2}
{'categoryId': 8189, 'title': '环球美食', 'type': 2}
{'categoryId': 8186, 'title': '服饰鞋靴', 'type': 2}
{'categoryId': 17236, 'title': '全球工厂店', 'type': 2}
{'categoryId': 15424, 'title': '网易严选', 'type': 2}
{'categoryId': 8191, 'title': '生鲜', 'type': 2}
{'categoryId': 14757, 'title': '宠物生活', 'type': 2}
{'categoryId': 14048, 'title': '考拉超市', 'type': 2}


In [221]:
import tablib
url = "http://dpp.boluome.com/https/generate"
tab_data = tablib.Dataset(headers=['一级分类','二级分类','三级分类','iconUrl'])
for category_1 in category_list:
    for category_2 in category_1['childCategoryViewList']:
        for category_3 in category_2['childCategoryViewList']:
            icon = category_3['iconUrl']
            os.system("wget {} -P /data/kaola/ -q".format(icon))
            new_icon = 'https://f1.otosaas.com/img/icon/kaola/'+ icon.split('/')[-1]
            response = requests.post(url,data={"https": new_icon},
                                     files={"image": open(f"/data/kaola/{icon.split('/')[-1]}", 'rb')}).json()

            tab_data.append([category_1['categoryName'],category_2['categoryName'],category_3['categoryName'],response['https']])
with open('kaola_category_tree.xlsx','wb') as f:
    f.write(tab_data.xlsx)